In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial as sfactorial
from copy import copy
from time import time
%matplotlib inline

In [6]:
x = 1
y = 1.
z = "Hi"

xlist = [1, 2, 3]
ylist = [1., 2., 3.]
zlist = ["Hi", "Howdy", "Hello"]

print(type(x))
print(type(y))
print(type(z))
print()
print(type(xlist))
print(type(ylist))
print(type(zlist))
print()
print(type(xlist[0]))
print(type(ylist[0]))
print(type(zlist[0]))

<class 'int'>
<class 'float'>
<class 'str'>

<class 'list'>
<class 'list'>
<class 'list'>

<class 'int'>
<class 'float'>
<class 'str'>


But since this is a scientific computation class, we have another type. It is the NumPy Array. You have seen this before with the command `np.linspace`. In more detail, when we write the assignment statement `xvals = np.linspace(a, b, n)`

we create an array of points `xvals[j]` such that 

`xvals[j] = a + (b-a)*j/(n-1), j=0, ..., n-1.`

Now, this part is important. Note that

In [7]:
xvals = np.linspace(-2., 2., int(1e1) + 1) 

print(type(xvals))
print(xvals)

<class 'numpy.ndarray'>
[-2.  -1.6 -1.2 -0.8 -0.4  0.   0.4  0.8  1.2  1.6  2. ]


So a NumPy array really is not just a Python list. And the difference is huge. In NumPy, I can do arithmetic on arrays. I cannot do arithmetic on Python lists. Let me show you what I mean.

In [8]:
print(type(xvals-xvals))
print(type(list(xvals)))
print(type(np.array(list(xvals))))
print(type(list(xvals)-list(xvals)))

<class 'numpy.ndarray'>
<class 'list'>
<class 'numpy.ndarray'>


TypeError: unsupported operand type(s) for -: 'list' and 'list'

# Vectorization in NumPY

We are now going to explore a key feature of NumPy which is *vectorization*. This is a feature whereby instead of explicitly calling a loop, NumPy just knows to iterate over every entry of a NumPY array in some reasonable manner. To get a feel for this, let's revisit our approximation for $\pi$ from the homework, where we used the code

In [9]:
def pi_approx(n):
    tot = 0.
    nsq = n**2.
    for kk in range(0, int(n)+1):
        tot += np.sqrt(nsq-kk**2.)
    
    return 4.*tot/nsq

to represent the formula $$ \pi = \lim_{n \rightarrow} \frac{4}{n^2} \sum_{k=0}^{n} \sqrt{n^{2}-k^{2}}$$
And we know that works, but again note the explicit `for` loop. To get far better performance, we make use of NumPy arrays to generate each. What we see here is that if we could first generate all the entries in the sum and then sum along that array of entries, we could probably spare ourselves some grief. This is exactly how vectorization in NumPy works. To wit then, we use the following code

In [10]:
def pi_approx_vec(n):
    kvals = np.arange(0, int(n)+1)
    nsq = n**2
    tot = np.sum(np.sqrt(nsq*np.ones(int(n)+1)-kvals**2.))
    return 4.*tot/nsq

So note the appearance of the NumPy array `kvals` and the use of the `np.sum()` function, which adds up entries along an array. Thus, we have written a version of the code that never calls a `for` explicitly. What do we then get for our efforts?

In [11]:
start = time()
pi_approx(1e7)
end = time()
print(end - start)


start = time()
pi_approx_vec(1e7)
end = time()
print(end - start)

8.618546962738037
0.13657760620117188


# Root Finding
Root finding refers to the general problem of searching for a solution of an equation $F(x)=0$ for some function $F(x)$. This is a very general problem and it comes up a lot in mathematics! For example, if we want to optimize a function $f(x)$ then we need to find critical points and therefore solve the equation $f^{\prime} =0 $.

There are few examples where there exist exact methods for finding solutions. For example, the quadratic formula $$ x = \frac{-b \pm \sqrt{b^{2}-4ac}}{2a} $$ gives us an exact method for finding roots of the equation $$ax^{2} + bx +c=0$$
There is a general formula to solve a cubic equation and even a quartic (degree 4) equation (but the formula is too complicated to be useful).

But there does not exist a formula for a quintic (degree 5) polynomial. And ther are many more examples of equations with no known method to solve them exactly.

What can we do? Use nuerical methods to find approximate solutions.

## Bisection Method
The simplest root finding algorith is the bisection method. The algorithm applies to any continuous function $f(x)$ on an interval $[a, b]$ where the value of the function $f(x)$ changes sign from $a \text{to} b$. The idea is simple: divide the interval in two, a solution must exisst within one subinterval, select the subinterval where the sign of $f(x)$ changes and repeat.

>**Criteria for Bisection Method:_** For the Bisection method to work on an interval $[a,b]$, we need $f$ to be continuous on $[a,b]$, and we need $f(a)f(b)<0$.

### Algorithm
The bisectoin method procedure is:
1. Choose a starting interval $[a_{0},b_{0}]$ such that $f(a_{0})f(b_{0})<0$.
1. Compute $f(m_{0})$ where $m_{0}=(a_{0}+b_{0})/2 is the midpoint.
1. Determine the next subinterval $[a_{1},b_{1}]$
    
    A. If $f(a_{0})f(m_{0}) <0$, then let $[a_{1},b_{1}]$ be the next interval with $a_{1}=a_{0}$ and $b_{1}=m_{0}$.
    
    B. If $f(b_{0})f(m_{0}) <0$, then let $[a_{1},b_{1}]$ be the next interval with $a_{1}=m_{0}$ and $b_{1}=b_{0}$.
1. Repeat (2) and (3) until the interval $[a_{N},b_{N}]$ reaches some predetermined length.
1. Return the midpoint value $m_{N} = (a_{N}+b_{N})/2$.

A solutoni of the equation $f(x)=0$ in the interval $[a,b]$ is guaranteed by the Intermediate Value Theorem provided $f(x)$ is continuous on $[a,b]$ and $f(a)f(b)<0$. In other words, the function changes sign over the interval and therefore must equal 0 at some point in ter interval $[a,b]$.

### Absolute Error 

The bisection method does not (in general) produce an exact solution of an equation $f(x) = 0$. However, we can give an estimate of the absolute error in the approximation.
* * *
**Theorem**. Let $f(x)$ be a continuous function on $[a,b]$ such that $f(a)f(b)<0$. After $N$ iterations of the bisection method, let $x_{N}$ be the midpoint in the $N$th subinterval $[a_{N},b_{N}]$ $$x_{N}=\frac{a_{N}+b_{N}}{2}$$ There exists an exact solution $x_{\text{true}}$ of the equation $f(x)=0$ in the subinterval $[a_{N},b_{N}]$ and the absolute error is $$ \|x_{\text{true}-x_{N}}\| \leq \frac{b-a}{2^{N+1}}$$
- - -
Note that we can rearrange the error bound to see the minimum number of iterations required to guarantee absolute error less than a prescribed $\epsilon$:
$$\frac{b-a}{2^{N+1}} < \epsilon$$ 
$$\frac{b-a}{\epsilon} < 2^{N+1}$$
$$\ln \left(\frac{b-a}{\epsilon}\right) < (N+1)\ln(2)$$
$$\frac{\ln\left(\frac{b-a}{\epsilon}\right)}{\ln(2)} -1 < N$$ 

### Implementation
Write a function called `bisection` which takes 4 input parameters `f`, `a`, `b` and `N` and returns the approximation of a solution of $f(x)=0$ given by $N$ iterations of the bisection method. If $f(a_{n})f(b_{n}) \geq 0$ at any point in the iteration (caused either by a bad initial interval or rounding error in computations), then print `"Bisection method fails.""` and return `None`.

In [12]:
def bisection(f, a, b, N):
    '''Approximate solution of f(x)=0 on interval [a,b] by bisection method.
    
    Parameters
    --------------
    f : function
        The function for which we are trying to approximate a solution f(x)=0.
    a, b: numbers
        The interval in which to search for a solution. The function returns None if f(a)*f(b) >= 0 since a solution is not guranteed.
    N : (positive) integer
        The number of iterations to implement
        
    Returns
    ---------
    x_N : number
        The midpoint of the Nth interval computed by the bisection method. The initial interval [a_0,b_0] is given by [a,b].
        If f(m_n) == 0 for some midpoint m_n = (a_n + b_n)/2, then the function returns this solution.
        If all signs of values f(a_n), f(b_n) are f(m_n) are the same at any iteration, the bisection method fails and return None.
        
    Examples
    ---------
    >>> f = lambda x: x**2 - x - 1
    >>> bisection(f, 1, 2, 25)
    1.61803~
    >>> f = lambda x: (2*x - 1)*(x - 3)
    >>> bisection(f, 0, 1, 10)
    0.5
    '''
    
    
    if f(a)*f(b) >= 0 :
        print("Bisection method fails.")
        return None
    
    a_n = a ; b_n = b
    
    for n in range(1, N+1):
        m_n = (a_n + b_n)/2
        f_m_n = f(m_n)
        if f(a_n)*f_m_n < 0:
            a_n = a_n ; b_n = m_n
            
        elif f(b_n)*f_m_n < 0:
            a_n = m_n; b_n = b_n
            
        elif f_m_n == 0:
            print("Found exact solution.")
            return m_n
        
        else:
            print("Bisection method fails.")
            return None
        
    return (a_n + b_n)/2

#### Lambda Functions
Lambda functions are small anonymous functions created with the `lambda` keyword:
    function_name = lambda parameter: return_value

The expression on the right side of the assignment operator `=` is an anonymous lambda function and we assign it to the variable name `fuction_name`.

Lambda functions are useful in at least two ways:
* Define a short function in a single line code
* Define a function within some other Python expression

For example, the function average defined above can be written in a single concise line:

In [13]:
average = lambda x: sum(x)/len(x)

The `lambda` keyword indicates that what follows is a function definition. The variable name `x` before the colon is the input parameter and the expression after the colon is the return value. Let's verify our lambda function returns the correct values:

In [14]:
average([1, 2, 3, 4])

2.5

Create lambda functions with several input parameters by listing variable names separated by commas. For example, let's create a function called `hypotenuse` which takes input parameters `x` and `y` and returns the length of the hypotenuse of the right angle triangle with sides `x` and `y`.

In [15]:
hypotenuse = lambda x, y : (x**2 + y**2) ** 0.5

In [16]:
hypotenuse(3, 4)

5.0

#### Examples

##### Golden Ratio
Let's use our function with input parameters $f(x) = x^{2} -x -1$ and $N=25$ iterations on $[1,2]$ to approximate the golden ratio $$\phi = \frac{1+\sqrt{5}}{2}$$
The golden ratio $\phi$ is a root of the quadratic polynomial $x^{2}-x-1=0$.

In [17]:
f = lambda x:x**2 - x -1
approx_phi = bisection(f, 1, 2, 25)
print(approx_phi)

1.618033990263939


The absolute error is guaranteed to be less than $(2-1)/2^{26}$ which is:

In [18]:
error_bound = 2**(-26)
print(error_bound)

1.4901161193847656e-08


Let's verify the absolute error is then tan this error bound:

In [20]:
abs( (1 + 5**0.5)/2 - approx_phi) < error_bound

True

## Secant Method
The secant method is very similar to the bisection method except instead of dividing each interval by choosing the midpoint the secant method divides each interval by the secant line connecting the endpoints. The secant method always converges to a root of $f(x)=0$ provided that $f(x)$ is continuous on $[a,b]$ and $f(a)f(b)<0$.

### Secant Line Formula
Let $f(x)$ be a continuous function on a closed interval $[a,b]$ such that $f(a)f(b)<0$. A solution of the equation $f(x)=0$ for $x \in [a,b]$ is guaranteed by the Intermediate Value Theorem. Consider the line connecting the endpoint values $(a, f(a))$ and $(b, f(b))$. The lin econnecting these two points is called the secant line and is given by the formula
$$ y = \frac{f(b)-f(a)}{b-a}(x-a) + f(a)$$
The point where the secant line crosses the $x$-axis is 
$$0=\frac{f(b)-f(a)|{b-a}}(x-a)+f(a)$$
which we solve for $x$ $$x=a-f(a)\frac{b-a}{f(b)-f(a)}$$

### Algorithm
The secant method procedure is almost identical to the bisection method. The only difference it how we divide each subinterval.
1. Choose a starting interval $[a_{0},b_{0}]$ such that $f(a_{0})f(b_{0})<0$.
1. Compute $f(x_{0})$ where $X_{0}$ is given by the secant line $$ x_{0} = a_{0} -f(a_{0})\frac{b_{0}-a_{0}}{f(b_{0})-f(a_{0})}$$
1. Determine the next subinterval $[a_{1},b_{1}]$:
    
    A. If $f(a_{0})f(x_{0}) <0$, then let $[a_{1},b_{1}]$ be the next interval with $a_{1}=a_{0}$ and $b_{1}=x_{0}$.
    
    B. If $f(b_{0})f(x_{0})<0$, then let $[a_{1},b_{1}]$ be the next interval with $a_{1}=x_{0}$ and $b_{1}=b_{0}$.
1. Repeat (2) and (3) until the interval $[a_{N},b_{N}]$ reaches some predetermined length.
1. Return the value $x_{n}$, the $x$-intercept of the $N$th subinterval.

A solution of the equation $f(x)=0$ in the interval $[a,b]$ is guaranteed by the Intermediate Value Theorem provided $f(x)$ is continuous on $[a,b]$ and $f(a)f(b)<0$. In other words, the function changes sign over the interval and therefore must equal 0 at some point in the interval $[a,b]$.

### Implementation
Write a function called `secant` which takes 4 input parameters `f`, `a`, `b` and `N` and returns the approximation of a solution of $f(x)=0$ given by `N` iterations of the secant method.
If $f(a_{n})f(b_{n}) \geq 0$ at any point in the iteration (caused either by a bad initial interval or rounding error in computations), then print `"Secant method fails."` and return `None`.

In [2]:
def secant(f, a, b, N):
    '''Approximate solution of f(x)=0 on interval [a,b] by the secant method.
    
    Parameters
    -------------
    f : function
        The function for which we are trying to approximate a solution f(x)=0.
    a,b : numbers
        The interval in which to search for a solution. the function returns None if f(a)*f(b) >= 0 sice a solution is not guaranteed.
        
    N : (positive) integer
        The number of iterations to implement.
        
    Returns
    ----------------
    m_N : number
        The x intercept of the secant line on the Nth interval
                m_n = a_n - f(a_n)*(b_n - a_n)/(f(b_n)-f(a_n))
        the initial interval [a_0,b_0] is given by [a,b]. If f(m_n) == 0
        for some intercept m_n then the function returns this solution.
        If all signs of values f(a_n), f(b_n) and f(m_n) are the same at any iterations, the secant method fails and return None.
        
    Examples 
    --------------
    >>> f = lambda x: x**2 - x - 1
    >>> secant(f,1,2,5)
    1.61802575~
    '''
    
    if f(a)*f(b) >= 0:
        print("Secant method fails")
        return None
    
    
    for n in range(1,N+1):
        x0 = a-f(a)*(b-a)/(f(b)-f(a))
        if f(x0) ==0:
            print("Fond exact solution", x0)
            break
            return x0 
        
        elif f(a)*f(x0) < 0:
            b = x0
        
        elif f(b)*f(x0) < 0:
            a = x0
            
        else:
            print("Secant Method fails")
            break
            return None
        
    return a - f(a)*(b-a)/(f(b)-f(a))

### Examples
#### Supergolden Ratio
Let's test our function with input values fo which we know the corrct output. Let's find an approximation of the supergolden ratio: the only real root of the polynomial $p(x)=x^{3}-x^{2}-1$.

In [4]:
p = lambda x : x**3 - x**2 - 1
print(p(1))
print(p(2))

-1
3


Since the polynomial changes sign in the interval $[1,2]$, we can apply the secant method with this as the starting interval:

In [5]:
approx = secant(p, 1, 2, 20)
print(approx)

1.4655712311394433


The exact value of the supergolden ratio is $$\frac{1+\sqrt[3]{\frac{29+3\sqrt{93}}{2}}+\sqrt[3]{\frac{29-3\sqrt{93}}{2}}}{3}$$

In [6]:
supergolden = (1+((29+3*93**0.5)/2) ** (1/3) + ((29-3*93**0.5)/2)**(1/3))/3
print(supergolden)

1.4655712318767682


Let's compare our approximation with the exact solution:

In [7]:
error = abs(supergolden - approx)
print(error)

7.373248678277378e-10


## Newton's Method
Newton's method is a root finding method that uses linear approximation. In particular, we guess a solution $x_{0}$ of the equation $f(x)=0$, compute the linear approximation of $f(x)$ at $x_{0}$ and then find the $x$-intercept of the linear approximation.

### Formula
Let $f(x)$ be a differentiable function. If $x_{0}$ is near a solution of $f(x)=0$ then we can approximate $f(x)$ by the tangent line at $x_{0}$ and compute the $x$-intercept of the tangent line.
The equatoin of the tangent line at $x_{0}$ is $$y=f^{\prime}(x_{0})(x-x_{0}) + f(x_{0})$$
The $x$-intercept is the solution $x_{1}$ of the equation $$0=f^{\prime}(x_{1}-x_{0})+f(x_{0})$$ and we solve for $x_{1}$
$$ x_{1}=x_{0} - \frac{f(x_{0})}{f^{\prime}(x_{0})}$$
If we implement this procedure repeatedly, then we obtain a sequence given by the recurvsive formula $$x_{n+1} = x_{n} - \frac{f(x_{n})}{f^{\prime}(x_{n})}$$
which (potentially) converges to a solution of the equation $f(x)=0$.

### Advantages/Disadvantages
When it converges, Newton's method usually converges very quickly and this is its main advantage. However, Newton's method is not guaranteed to converge and this is obviously a big disadvantage especially compared to the bisection and secant methods which are guaranteed to converge to a solution (provided they start with an interval containing a root).

Newton's method also requires computing values of the derivative fo the function in questoin. This is potentially a disadvantage if the derivative is difficult to compute.

The stopping criteria for Newton's method differs from the bisection and secant methods. In those methods, we know how close we are to a solution because we are computing intervals which contain a solution. In Newton's method, we don't know how close we are to solution. All we can compute is the value $f(x)$ and so we implement a stopping criteria based on $f(x)$.

Finally, there's no guarantee that the method converges to a solution and we should set a maximum number of iterations so that our implementation ends if we don't find a solutoion.

### Implementation
Let's write a function called `newton` which takes 5 input parameters `f`, `Df`, `x0`, `epsilon` and `max_iter` and returns an approximation of a solutoin of $f(x)=0$ by Newton's method. The function may terminate in 3ways:
1. If `abs(f(xn)) < epsilon`, the algorithm has found an approximate solution and returns `xn`.
1. If `f'(xn) == 0`, the algorithm sotps and reutns `None`.
1. If the number of iterations exceed `max_iter`, the algorithm stops and returns `None`.

In [8]:
def newton(f, Df, x0, epsilon, max_iter):
    '''Approximate solution of f(x)=0 by Newton's method.
    
    Parameters
    -----------
    f : function
        Function for which we are searching for a solutoin f(x)=0.
    Df : function
        Derivative of f(x).
    x0 : number
        Initial guess for a solution f(x)=0.
    epsilon : number 
        Stopping criteria is abs(f(x)) < epsilon. 
    max_iter : integer
        Maximum number of iterations of Newton's method.
        
    Returns
    -------------
    xn : number
        Implement Newton's method: compute the linear approximation
        of f(x) at xn and find x intercept by the formula
            x = xn -f(xn)/Df(xn)
        Continue until abs(f(xn)) < epsilon and return xn.
        If Df(xn) == 0, return None. If the number of iterations
        exceeds max_iter, then return None.
        
    Examples
    ---------
    >>> f = lambda x:x**2 - x - 1
    >>> Df = lambda x: 2*x -1
    >>> newton(f, Df, 1, 1e-8, 10)
    Found solutoin after 5 iterations.
    1.6180~
    '''
    
    xn = x0
    for n in range(0, max_iter):
        fxn = f(xn)
        if abs(fxn) < epsilon:
            print("Found solution after", n, "iterations.")
            return xn
        Dfxn = Df(xn)
        if Dfxn == 0:
            print("Zero derivative. No solution found.")
            return None
        
        xn -= fxn/Dfxn
    print("Exceed maximum iterations. No solution found.")
    return None

#### Examples
##### Supergolden Ratio
Let's test our function `newton` on the polynomial $p(x)=x^{3}-x^{2}-1$ to approximate the super golden ratio.

In [10]:
p = lambda x: x**3 -x**2 - 1
Dp = lambda x: 3*x**2 - 2*x
approx = newton(p, Dp, 1, 1e-10, 10)
print(approx)

Found solution after 6 iterations.
1.4655712318767877


How many iterations of the bisection method starting with the interval $[1,2]$ can achieve the smae accuaracy?

#### Divgerent Example
Newton's method diverges in certain cases. For example, if the tangent line at the root is vertical as in $f(x)=x^{1/3}$. Note that bisection and secant methods would converge in this case.

In [11]:
f = lambda x: x**(1/3)
Df = lambda x: (1/3)*x**(-2/3)
approx = newton(f, Df, 0.1, 1e-2, 100)

Exceed maximum iterations. No solution found.


### Convergence Rate
We now want a means of figuring out how fast the Bisection Method runs. So, if you think about it, at every iteration of the method, an approximation, say $c_{n}$, to the root, say $c_{*}$, is generated. For the method to converge, we mean that $$\lim_{n \rightarrow \infty}c_{n} = c_{*}$$, or equivalently $$\lim_{n \rightarrow \infty}\|c_{n}-c_{*}\| = 0$$.
But the question then becomes, how quickly does this limit go to zero?
*Problem*: Can you do this for the Bisection Method? In other words, is there a formula you can write down which tells you how quickly $\|c_{n}-c_{*}\| goes to zero?

In general, we think of answering this question by defining what is called the rate of convergence.
> **Rate of Convergence:** For an iterative sequence $c_{n} \rightarrow c_{*}$, we define the rate of convergence, $\alpha$, to be $$\lim_{n \rightarrow \infty}\frac{\|c_{n+1}-c_{*}\|}{\|c_{n}-c_{*}\|^{\alpha}} = \lambda $$

The idea here is that for very large $n$, we have that $$\|c_{n+1}-c_{*}\| \approx \lambda\|c_{n}-c_{*}\|^{\alpha}$$

*Problem*: What would a lagarithm tell you? How would you use that to numerically compute the rate of convergence?

*Problem*: Modify your code for the Bisection Method to find the rate of convergence. Does it agree with your theoretical prediction?

#### Exercises
**Exercise 1.** Let $p(x)=x^{3}-x-1$. The only real root of $p(x)$ is called the plastic number and is givnen by $$\frac{\sqrt[3]{108+12\sqrt{69}}+\sqrt[3]{108-12\sqrt{69}}}{6}$$